In [ ]:
#| default_exp handlers.maris_dump

# MARIS dump
> Data pipeline (handler) to convert global MARIS db dump into `NetCDF` format

The input data is a dump from already imported MARIS datasets.

**Dev. board**: https://trello.com/b/IszgV1bj/marisco

Questions:

* filtering status? 

## Packages import

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
from tqdm import tqdm
from pathlib import Path
import fastcore.all as fc
import pandas as pd
import numpy as np

from marisco.callbacks import (Callback, Transformer, SanitizeLonLatCB, EncodeTimeCB)
from marisco.metadata import (GlobAttrsFeeder, BboxCB,
                              DepthRangeCB, TimeRangeCB,
                              ZoteroCB, KeyValuePairCB)
from marisco.configs import lut_path, cdl_cfg, cfg, nc_tpl_path, Enums
from marisco.serializers import NetCDFEncoder

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# | export
fname_in = Path().home() / 'pro/data/maris/MARIS_exportSample_20240313.txt'
dir_dest = '../../_data/output/dump'

## Utils

In [ ]:
# | export
def load_dump(fname): 
    return pd.read_csv(fname, sep='\t', encoding='ISO-8859-1')   

def load_data(df:pd.DataFrame, # MARIS global dump 
                 ref_id:int, # Reference id of interest
                 ):
    "Load specific MARIS dataset through its ref_id"
    lut = {
        'Sediment': 'sediment',
        'Seawater': 'seawater',
        'Suspended matter': 'suspended-matter',
        'Biota': 'biota'}
    dfs = {}
    for name, grp in df[df.ref_id  == ref_id].groupby('samptype'): 
        dfs[lut[name]] = grp
    return dfs

def get_zotero_key(dfs):
    return dfs[next(iter(dfs))][['zoterourl']].iloc[0].values[0].split('/')[-1]

def get_fname(dfs):
    id, name = dfs[next(iter(dfs))][['ref_id', 'displaytext']].iloc[0]
    name = name.replace(',', '').replace('.', '').replace('-', ' ').split(' ')
    return '-'.join(([str(id)] + name)) + '.nc'

## Load data

In [ ]:
df = load_dump(fname_in)

In [ ]:
len(df.ref_id.unique())

526

In [ ]:
df.columns

Index(['ref_id', 'displaytext', 'samptype', 'nuclide_id', 'latitude',
       'longitude', 'begperiod', 'endperiod', 'sampdepth', 'totdepth',
       'uncertaint', 'unit_id', 'detection', 'area_id', 'species_id',
       'biogroup_id', 'bodypar_id', 'sedtype_id', 'volume', 'salinity',
       'temperatur', 'sampmet_id', 'prepmet_id', 'counmet_id', 'activity',
       'zoterourl'],
      dtype='object')

In [ ]:
df.biogroup_id.unique()

array([4, 0, 2, 13, 14, 3, 12, 11, 10, 5, 1, 7, 8, 9, 6, '11', '0', '14',
       '13', '2', '4', 'biogroup_id', '12', '10', '3', '5', '7', '1', '8'],
      dtype=object)

In [ ]:
df.head()

,ref_id,displaytext,samptype,nuclide_id,latitude,longitude,begperiod,endperiod,sampdepth,totdepth,...,bodypar_id,sedtype_id,volume,salinity,temperatur,sampmet_id,prepmet_id,counmet_id,activity,zoterourl
0,182,"Urban et al., 2015",Biota,33,-35.140833,117.604444,2014-05-06 00:00:00,NaN,-1.0,NaN,...,52,0,NaN,NaN,NaN,0,6,20,0.387,https://www.zotero.org/groups/2432820/maris/it...
1,182,"Urban et al., 2015",Biota,47,-35.140833,117.604444,2014-05-06 00:00:00,NaN,-1.0,NaN,...,52,0,NaN,NaN,NaN,0,6,5,1.44,https://www.zotero.org/groups/2432820/maris/it...
2,182,"Urban et al., 2015",Biota,31,-16.466944,123.535833,2014-02-27 00:00:00,NaN,-1.0,NaN,...,52,0,NaN,NaN,NaN,0,6,20,0.042,https://www.zotero.org/groups/2432820/maris/it...
3,182,"Urban et al., 2015",Biota,33,-16.466944,123.535833,2014-02-27 00:00:00,NaN,-1.0,NaN,...,52,0,NaN,NaN,NaN,0,6,20,0.075,https://www.zotero.org/groups/2432820/maris/it...
4,182,"Urban et al., 2015",Biota,47,-16.466944,123.535833,2014-02-27 00:00:00,NaN,-1.0,NaN,...,52,0,NaN,NaN,NaN,0,6,5,0.069,https://www.zotero.org/groups/2432820/maris/it...


In [ ]:
# 52, 191 (OSPAR), 100 (HELCOM), 717 (only seawater)
ref_id = 100

In [ ]:
dfs = load_data(df, ref_id)

In [ ]:
dfs.keys()

dict_keys(['biota', 'seawater', 'sediment'])

## Data transformation pipeline

### Normalize nuclide names

In [ ]:
#| export
def get_varnames_lut():
    fname = lut_path() / 'dbo_nuclide.xlsx'
    df_nuclide = pd.read_excel(fname, usecols=['nuclide_id', 'nc_name'])
    return df_nuclide.set_index('nuclide_id').to_dict()['nc_name']

In [ ]:
# | export
class RemapRdnNameCB(Callback):
    "Remap to MARIS radionuclide names."
    def __init__(self,
                 fn_lut=get_varnames_lut):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            tfm.dfs[k]['nuclide_id'] = tfm.dfs[k]['nuclide_id'].replace(lut)

In [ ]:
dfs = load_data(df, ref_id)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[RemapRdnNameCB()])

print(tfm()['sediment']['nuclide_id'].unique())

['ru106' 'sb125' 'cs134' 'cs137' 'k40' 'co60' 'ag110m' 'ra226' 'th232'
 'pb212' 'pb214' 'pu238' 'am241' 'pu239_240_tot' 'zr95' 'mn54' 'ac228'
 'u235' 'tl208' 'be7' 'bi214' 'ra223' 'ru103' 'sr90' 'eu155' 'ba140'
 'co58' 'ra224' 'po210' 'ra228' 'th228' 'ce144' 'cs134_137_tot' 'pb210'
 'pu239' 'cd109' 'bi212' 'pu238_240_tot' 'nb95' 'ir192' 'sb124' 'zn65'
 'th234' 'pu241']


### Rename columns

In [ ]:
dfs['sediment'].columns

Index(['ref_id', 'displaytext', 'samptype', 'nuclide_id', 'latitude',
       'longitude', 'begperiod', 'endperiod', 'sampdepth', 'totdepth',
       'uncertaint', 'unit_id', 'detection', 'area_id', 'species_id',
       'biogroup_id', 'bodypar_id', 'sedtype_id', 'volume', 'salinity',
       'temperatur', 'sampmet_id', 'prepmet_id', 'counmet_id', 'activity',
       'zoterourl'],
      dtype='object')

In [ ]:
#| export
# To be added: endperiod, lab
def renaming_rules():
    vars = cdl_cfg()['vars']
    # Define column names renaming rules
    return {
        'latitude': vars['defaults']['lat']['name'],
        'longitude': vars['defaults']['lon']['name'],
        'begperiod': vars['defaults']['time']['name'],
        'sampdepth': vars['defaults']['smp_depth']['name'],
        'totdepth': vars['defaults']['tot_depth']['name'],
        'uncertaint': vars['suffixes']['uncertainty']['name'],
        'unit_id': vars['suffixes']['unit']['name'],
        'detection': vars['suffixes']['detection_limit']['name'],
        'area_id': vars['defaults']['area']['name'], 
        'species_id': vars['bio']['species']['name'],
        'biogroup_id': vars['bio']['bio_group']['name'],
        'bodypar_id': vars['bio']['body_part']['name'],
        'sedtype_id': vars['sed']['sed_type']['name'],
        'volume': vars['suffixes']['volume']['name'],
        'salinity': vars['suffixes']['salinity']['name'],
        'temperatur': vars['suffixes']['temperature']['name'],
        'sampmet_id': vars['suffixes']['sampling_method']['name'],
        'prepmet_id': vars['suffixes']['preparation_method']['name'],
        'counmet_id': vars['suffixes']['counting_method']['name'],
        'activity': 'value',
        'nuclide_id': 'nuclide'
    }

In [ ]:
#| export
class RenameColumnCB(Callback):
    "Renaming variables to MARIS standard names."
    def __init__(self, renaming_rules=renaming_rules): fc.store_attr()
    def __call__(self, tfm):
        lut = renaming_rules()
        coi = lut.keys()
        for k in tfm.dfs.keys():
            # Select cols of interest
            tfm.dfs[k] = tfm.dfs[k].loc[:, coi]
            # Rename cols
            tfm.dfs[k].rename(columns=lut, inplace=True)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB()
    ])

print(tfm()['sediment'])

               lat        lon                 time smp_depth tot_depth  \
549778   54.838333        9.9  1989-06-14 00:00:00      -1.0      24.0   
549779   54.838333        9.9  1989-06-14 00:00:00      -1.0      24.0   
549780   54.838333        9.9  1989-06-14 00:00:00      -1.0      24.0   
549781   54.838333        9.9  1989-06-14 00:00:00      -1.0      24.0   
549782   54.838333        9.9  1989-06-14 00:00:00      -1.0      24.0   
...            ...        ...                  ...       ...       ...   
1532415  57.619722  23.621389  2005-12-02 00:00:00      -1.0      55.0   
1532416  57.619722  23.621389  2005-12-02 00:00:00      -1.0      55.0   
1532417  57.619722  23.621389  2005-12-02 00:00:00      -1.0      55.0   
1532418  57.619722  23.621389  2005-12-02 00:00:00      -1.0      55.0   
1532419  57.619722  23.621389  2005-12-02 00:00:00      -1.0      55.0   

             _unc _unit _dl  area species  ... body_part sed_type _vol _sal  \
549778       3.99     4   =  237

### Drop NaN only columns

In [ ]:
#| export
class DropNAColumnsCB(Callback):
    "Drop variable containing only NaN or 'Not available' (id=0 in MARIS lookup tables)."
    def __init__(self, na_value=0):
        fc.store_attr()
        
    def isMarisNA(self, col): 
        return len(col.unique()) == 1 and col.iloc[0] == self.na_value
    
    def dropMarisNA(self, df):
        na_cols = [col for col in df.columns if self.isMarisNA(df[col])]
        return df.drop(labels=na_cols, axis=1)
        
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k] = tfm.dfs[k].dropna(axis=1, how='all')
            tfm.dfs[k] = self.dropMarisNA(tfm.dfs[k])

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB()
    ])

print(tfm()['sediment'])

               lat        lon                 time smp_depth tot_depth  \
549778   54.838333        9.9  1989-06-14 00:00:00      -1.0      24.0   
549779   54.838333        9.9  1989-06-14 00:00:00      -1.0      24.0   
549780   54.838333        9.9  1989-06-14 00:00:00      -1.0      24.0   
549781   54.838333        9.9  1989-06-14 00:00:00      -1.0      24.0   
549782   54.838333        9.9  1989-06-14 00:00:00      -1.0      24.0   
...            ...        ...                  ...       ...       ...   
1532415  57.619722  23.621389  2005-12-02 00:00:00      -1.0      55.0   
1532416  57.619722  23.621389  2005-12-02 00:00:00      -1.0      55.0   
1532417  57.619722  23.621389  2005-12-02 00:00:00      -1.0      55.0   
1532418  57.619722  23.621389  2005-12-02 00:00:00      -1.0      55.0   
1532419  57.619722  23.621389  2005-12-02 00:00:00      -1.0      55.0   

             _unc _unit _dl  area sed_type _sampmet _prepmet    value nuclide  
549778       3.99     4   =  23

### Sanitize detection limit values

In [ ]:
#| export
def get_dl_lut():
    fname = lut_path() / 'dbo_detectlimit.xlsx'
    df_nuclide = pd.read_excel(fname, usecols=['name', 'id'])
    return df_nuclide.set_index('name').to_dict()['id']

In [ ]:
get_dl_lut()

{'Not applicable': -1, 'Not Available': 0, '=': 1, '<': 2, 'ND': 3, 'DE': 4}

In [ ]:
#| export
class SanitizeDetectionLimitCB(Callback):
    "Assign Detection Limit name to its id based on MARIS nomenclature."
    def __init__(self,
                 fn_lut=get_dl_lut):
        fc.store_attr()
        self.var_name = cdl_cfg()['vars']['suffixes']['detection_limit']['name']

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            tfm.dfs[k][self.var_name] = tfm.dfs[k][self.var_name].replace(lut)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB()
    ])

print(tfm()['sediment']['_dl'])

549778     1
549779     1
549780     1
549781     1
549782     1
          ..
1532415    1
1532416    1
1532417    1
1532418    1
1532419    1
Name: _dl, Length: 123196, dtype: int64


### Parse time

In [ ]:
#| export
class ParseTimeCB(Callback):
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k]['time'] = pd.to_datetime(tfm.dfs[k].time, format='ISO8601')

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB()
    ])

print(tfm()['sediment'])

               lat        lon       time smp_depth tot_depth       _unc _unit  \
549778   54.838333        9.9 1989-06-14      -1.0      24.0    3.99000     4   
549779   54.838333        9.9 1989-06-14      -1.0      24.0        NaN     2   
549780   54.838333        9.9 1989-06-14      -1.0      24.0    1.67400     4   
549781   54.838333        9.9 1989-06-14      -1.0      24.0        NaN     2   
549782   54.838333        9.9 1989-06-14      -1.0      24.0    1.82900     4   
...            ...        ...        ...       ...       ...        ...   ...   
1532415  57.619722  23.621389 2005-12-02      -1.0      55.0   86.28360     4   
1532416  57.619722  23.621389 2005-12-02      -1.0      55.0        NaN     2   
1532417  57.619722  23.621389 2005-12-02      -1.0      55.0   24.45552     4   
1532418  57.619722  23.621389 2005-12-02      -1.0      55.0        NaN     2   
1532419  57.619722  23.621389 2005-12-02      -1.0      55.0  123.25680     4   

        _dl  area sed_type 

### Reshape: long to wide

In [ ]:
#| export
class ReshapeLongToWide(Callback):
    "Convert data from long to wide with renamed columns."
    def __init__(self, columns='nuclide', values=['value']):
        fc.store_attr()
        # Retrieve all possible derived vars (e.g 'unc', 'dl', ...) from configs
        self.derived_cols = [value['name'] for value in cdl_cfg()['vars']['suffixes'].values()]
    
    def renamed_cols(self, cols):
        "Flatten columns name"
        return [inner if outer == "value" else f'{inner}{outer}'
                if inner else outer
                for outer, inner in cols]

    def pivot(self, df):
        # Among all possible 'derived cols' select the ones present in df
        derived_coi = [col for col in self.derived_cols if col in df.columns]
        
        df.reset_index(names='sample', inplace=True)
        
        idx = list(set(df.columns) - set([self.columns] + derived_coi + self.values))
        return df.pivot_table(index=idx,
                              columns=self.columns,
                              values=self.values + derived_coi,
                              fill_value=np.nan,
                              aggfunc=lambda x: x
                              ).reset_index()

    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k] = self.pivot(tfm.dfs[k])
            tfm.dfs[k].columns = self.renamed_cols(tfm.dfs[k].columns)

In [ ]:
# How do that work?
#    In our case, unique idx is a composite index (lon, lat, time, depth)
#    We want to transform:
# idx , nuc, val
# 1   , a  , 1
# 1   , b  , 2
# 2   , c  , 3

# to: 
# idx, a,  b,   c
# 1  , 1,  2,   nan
# 2  ,nan, nan, 3 
df_test = pd.DataFrame({
    'idx': [1, 1, 2],
    'nuclide': ['a', 'b', 'c'],
    'value': [1, 2, 3]
    })

df_test.pivot_table(index='idx',
                    columns='nuclide',
                    values='value',
                    fill_value=np.nan,
                    # aggfunc=lambda x: x
                    )

nuclide,a,b,c
idx,,,
1,1.0,2.0,NaN
2,NaN,NaN,3.0


In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    ReshapeLongToWide()
    ])

print(tfm()['sediment'])

              lon  area       time  tot_depth   sample        lat  smp_depth  \
0        9.633333  2374 1987-04-08       16.0   549834  54.850000       -1.0   
1        9.633333  2374 1987-04-08       16.0   549835  54.850000       -1.0   
2        9.633333  2374 1987-04-08       16.0   549836  54.850000       -1.0   
3        9.633333  2374 1987-04-08       16.0   549837  54.850000       -1.0   
4        9.633333  2374 1987-04-08       16.0   549838  54.850000       -1.0   
...           ...   ...        ...        ...      ...        ...        ...   
122829  29.833333  2407 2005-10-02        0.0  1518808  59.983333       -1.0   
122830  29.833333  2407 2005-10-02        0.0  1518809  59.983333       -1.0   
122831  29.833333  2407 2005-10-02        0.0  1518810  59.983333       -1.0   
122832  29.833333  2407 2005-10-02        0.0  1528756  59.983333       -1.0   
122833  29.833333  2407 2005-10-02        0.0  1528757  59.983333       -1.0   

        sed_type ac228_dl ag110m_dl  ..

### Encode time (seconds since ...)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    ReshapeLongToWide(),
    EncodeTimeCB(cfg())
    ])

print(tfm()['sediment'])

              lon  area        time  tot_depth   sample        lat  smp_depth  \
0        9.633333  2374   544838400       16.0   549834  54.850000       -1.0   
1        9.633333  2374   544838400       16.0   549835  54.850000       -1.0   
2        9.633333  2374   544838400       16.0   549836  54.850000       -1.0   
3        9.633333  2374   544838400       16.0   549837  54.850000       -1.0   
4        9.633333  2374   544838400       16.0   549838  54.850000       -1.0   
...           ...   ...         ...        ...      ...        ...        ...   
122829  29.833333  2407  1128211200        0.0  1518808  59.983333       -1.0   
122830  29.833333  2407  1128211200        0.0  1518809  59.983333       -1.0   
122831  29.833333  2407  1128211200        0.0  1518810  59.983333       -1.0   
122832  29.833333  2407  1128211200        0.0  1528756  59.983333       -1.0   
122833  29.833333  2407  1128211200        0.0  1528757  59.983333       -1.0   

        sed_type ac228_dl a

### Sanitize coordinates

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    ReshapeLongToWide(),
    EncodeTimeCB(cfg()),
    SanitizeLonLatCB()
    ])

# print(tfm()['sediment'])
df_debug = tfm()['sediment']

## Encode to NetCDF

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    ReshapeLongToWide(),
    EncodeTimeCB(cfg()),
    SanitizeLonLatCB()
    ])

dfs_tfm = tfm()

In [ ]:
tfm.logs

['Remap to MARIS radionuclide names.',
 'Renaming variables to MARIS standard names.',
 "Drop variable containing only NaN or 'Not available' (id=0 in MARIS lookup tables).",
 'Assign Detection Limit name to its id based on MARIS nomenclature.',
 'Convert data from long to wide with renamed columns.',
 'Encode time as `int` representing seconds since xxx',
 'Drop row when both longitude & latitude equal 0. Drop unrealistic longitude & latitude values. Convert longitude & latitude `,` separator to `.` separator.']

In [ ]:
#| export
kw = ['oceanography', 'Earth Science > Oceans > Ocean Chemistry> Radionuclides',
      'Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure',
      'Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments',
      'Earth Science > Oceans > Ocean Chemistry, Earth Science > Oceans > Sea Ice > Isotopes',
      'Earth Science > Oceans > Water Quality > Ocean Contaminants',
      'Earth Science > Biological Classification > Animals/Vertebrates > Fish',
      'Earth Science > Biosphere > Ecosystems > Marine Ecosystems',
      'Earth Science > Biological Classification > Animals/Invertebrates > Mollusks',
      'Earth Science > Biological Classification > Animals/Invertebrates > Arthropods > Crustaceans',
      'Earth Science > Biological Classification > Plants > Macroalgae (Seaweeds)']

In [ ]:
#| export
def get_attrs(tfm, zotero_key, kw=kw):
    return GlobAttrsFeeder(tfm.dfs, cbs=[
        BboxCB(),
        DepthRangeCB(),
        TimeRangeCB(cfg()),
        ZoteroCB(zotero_key, cfg=cfg()),
        KeyValuePairCB('keywords', ', '.join(kw)),
        KeyValuePairCB('publisher_postprocess_logs', ', '.join(tfm.logs))
        ])()

In [ ]:
get_attrs(tfm, zotero_key='3W354SQG', kw=kw)

{'geospatial_lat_min': '30.435833333333335',
 'geospatial_lat_max': '65.75',
 'geospatial_lon_min': '9.633333333333333',
 'geospatial_lon_max': '53.5',
 'geospatial_bounds': 'POLYGON ((9.633333333333333 53.5, 30.435833333333335 53.5, 30.435833333333335 65.75, 9.633333333333333 65.75, 9.633333333333333 53.5))',
 'time_coverage_start': '1984-01-10T00:00:00',
 'time_coverage_end': '2018-12-14T00:00:00',
 'title': 'Radioactivity Monitoring of the Irish Marine Environment 1991 and 1992',
 'summary': '',
 'creator_name': '[{"creatorType": "author", "firstName": "A.", "lastName": "McGarry"}, {"creatorType": "author", "firstName": "S.", "lastName": "Lyons"}, {"creatorType": "author", "firstName": "C.", "lastName": "McEnri"}, {"creatorType": "author", "firstName": "T.", "lastName": "Ryan"}, {"creatorType": "author", "firstName": "M.", "lastName": "O\'Colmain"}, {"creatorType": "author", "firstName": "J.D.", "lastName": "Cunningham"}]',
 'keywords': 'oceanography, Earth Science > Oceans > Ocean 

In [ ]:
#| export
def enums_xtra(tfm, vars):
    "Retrieve a subset of the lengthy enum as 'species_t' for instance"
    enums = Enums(lut_src_dir=lut_path(), cdl_enums=cdl_cfg()['enums'])
    xtras = {}
    for var in vars:
        unique_vals = tfm.unique(var)
        if unique_vals.any():
            xtras[f'{var}_t'] = enums.filter(f'{var}_t', unique_vals)
    return xtras

In [ ]:
#| export
def encode(fname_in, fname_out, nc_tpl_path, **kwargs):
    df = load_dump(fname_in)
    ref_ids = kwargs.get('ref_ids', df.ref_id.unique())
    print('Encoding ...')
    for ref_id in tqdm(ref_ids, leave=False):
        dfs = load_data(df, ref_id)
        print(get_fname(dfs))
        tfm = Transformer(dfs, cbs=[
            RemapRdnNameCB(),
            RenameColumnCB(),
            DropNAColumnsCB(),
            SanitizeDetectionLimitCB(),
            ParseTimeCB(),
            ReshapeLongToWide(),
            EncodeTimeCB(cfg()),
            SanitizeLonLatCB(verbose=True)
            ])
       
        tfm()
        encoder = NetCDFEncoder(tfm.dfs, 
                                src_fname=nc_tpl_path,
                                dest_fname=Path(fname_out) / get_fname(dfs), 
                                global_attrs=get_attrs(tfm, zotero_key=get_zotero_key(dfs), kw=kw),
                                verbose=kwargs.get('verbose', False),
                                enums_xtra=enums_xtra(tfm, vars=['species', 'body_part'])
                                )
        encoder.encode()

### Single dataset

In [ ]:
tfm = Transformer(dfs, cbs=[
            RemapRdnNameCB(),
            RenameColumnCB(),
            DropNAColumnsCB(),
            SanitizeDetectionLimitCB(),
            ParseTimeCB(),
            ReshapeLongToWide(),
            EncodeTimeCB(cfg()),
            SanitizeLonLatCB(verbose=True)
            ])
dfs_test = tfm()

In [ ]:
dfs_test['biota'].be7_dl.unique()

array([nan,  1.,  2.])

In [ ]:
ref_id = 100
encode(fname_in, dir_dest, nc_tpl_path(), verbose=False, ref_ids=[ref_id])

Encoding ...


  0%|                                                                                                                                                                                 | 0/1 [00:00<?, ?it/s]

100-HELCOM-MORS-2018.nc


### All datasets

In [ ]:
encode(fname_in, dir_dest, nc_tpl_path(), verbose=False)

Encoding ...


  0%|          | 0/526 [00:00<?, ?it/s]

182-Urban-et-al-2015.nc


  0%|          | 1/526 [00:02<24:30,  2.80s/it]

183-Bokor-et-al-2016.nc


...

270-Garcia-Orellana-et-al-2016.nc


 88%|████████▊ | 461/526 [28:02<03:09,  2.92s/it]

294-Bode-et-al-2015.nc


697-Long-et-al-1998.nc


TypeError: ufunc 'points' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''